In [ ]:
# define encoding for blood types
blood_type_encoding = {
    'o-': 0,
    'o+': 1,
    'a-': 2,
    'a+': 3,
    'b-': 4,
    'b+': 5,
    'ab-': 6,
    'ab+': 7
}

#       Donor: o- o+ a- a+ b- b+ ab- ab+
truth_table = [[1,0,0,0,0,0,0,0],  # o-  recipients
               [1,1,0,0,0,0,0,0],  # o+
               [1,0,1,0,0,0,0,0],  # a-
               [1,1,1,1,0,0,0,0],  # a+
               [1,0,0,0,1,0,0,0],  # b-
               [1,1,0,0,1,1,0,0],  # b+
               [1,0,1,0,1,0,1,0],  # ab-
               [1,1,1,1,1,1,1,1]]  # ab+

def can_donate(donor, recipient):
    donor_index = blood_type_encoding[donor]
    recipient_index = blood_type_encoding[recipient]
    return truth_table[recipient_index][donor_index]

In [19]:
blood_type_encoding_yao = {
    'o-': (0,0,0),
    'o+': (0,0,1),
    'a-': (0,1,0),
    'a+': (0,1,1),
    'b-': (1,0,0),
    'b+': (1,0,1),
    'ab-': (1,1,0),
    'ab+': (1,1,1)
}

import random, os, hashlib
Label_Bytes = 16 # 128 bits

def G(A: bytes, B: bytes, gate_index: int) -> bytes:
    """
    PRF G : {0,1}^k x {0,1}^k x int -> {0,1}^{2k}
    k = 16 bytes = 128 bits
    """
    h = hashlib.sha256()
    h.update(A)
    h.update(B)
    h.update(gate_index.to_bytes(4, 'big'))
    digest = h.digest()
    return digest[:2*Label_Bytes]  # return first 32 bytes (256 bits)


In [ ]:
from typing import Dict, Tuple
import itertools

class GarbledCircuit:
    def __init__(self):
        self.input_wires = list(range(6))               # 6 Input bits, 0-2 from Alice, 3-5 from Bob
        self.output_wire = 6                            # Wire 6 for output
        self.garbled_table = []                         # to be filled with garbled entries
        self.keys: Dict[int, Tuple[bytes, bytes]] = {}  # wire index -> (label0, label1), decode key is last in dict
        self.gates = []                                 # list of gates (gate_type, input_wire1, input_wire2, output_wire)
        # Add more attributes as needed
    
    def generate(self):
        # gen random keys for input wires
        for wire in self.input_wires:
            k0 = os.urandom(Label_Bytes)
            k1 = os.urandom(Label_Bytes)
            self.keys[wire] = (k0, k1)

        z0 = os.urandom(Label_Bytes)
        z1 = os.urandom(Label_Bytes)
        self.keys[self.output_wire] = (z0, z1)

        # Build garbled table for the truth table
        entries = []
        gate_index = 0
        for bits in itertools.product([0,1], repeat=6):
            donor_bits = bits[0:3]
            recipient_bits = bits[3:6]
        pass

    def Generate(circuit):
    # Output the garbled table and keys
        pass

    def Encode(input, keys):
    # Output the encoded input labels
        pass

    def Evaluate(garbled_table, encoded_input):
    # Output the encoded output label
        pass

    def Decode(encoded_output, decode_key):
    # Output the final output bit
        pass



In [ ]:
# Class Garbled had G (generate), E (encode), Ev (evaluate), D (decode)

In [20]:
from typing import Dict, Tuple
import pprint

def is_compatible_pair(d_code: int, r_code: int) -> int:
    d_abo = d_code >> 1
    d_rh = d_code & 1
    r_abo = r_code >> 1
    r_rh = r_code & 1

    abo_ok = (
        (d_abo == 0) # O can donate to anyone
        or (d_abo == r_abo) # same type
        or (d_abo == 1 and r_abo == 3) # A can donate to AB
        or (d_abo == 2 and r_abo == 3) # B can donate to AB
    )

    rh_ok = (d_rh == 0) or (d_rh == r_rh) # Rh- can donate to anyone, Rh+ only to Rh+

    return int(abo_ok and rh_ok)

compat_table: Dict[Tuple[str, str], int] = {}
for d in range(8):
    for r in range(8):
        compat_table[(list(blood_type_encoding_yao.keys())[d], list(blood_type_encoding_yao.keys())[r])] = is_compatible_pair(d, r)

pprint.pprint(compat_table)



{('a+', 'a+'): 1,
 ('a+', 'a-'): 0,
 ('a+', 'ab+'): 1,
 ('a+', 'ab-'): 0,
 ('a+', 'b+'): 0,
 ('a+', 'b-'): 0,
 ('a+', 'o+'): 0,
 ('a+', 'o-'): 0,
 ('a-', 'a+'): 1,
 ('a-', 'a-'): 1,
 ('a-', 'ab+'): 1,
 ('a-', 'ab-'): 1,
 ('a-', 'b+'): 0,
 ('a-', 'b-'): 0,
 ('a-', 'o+'): 0,
 ('a-', 'o-'): 0,
 ('ab+', 'a+'): 0,
 ('ab+', 'a-'): 0,
 ('ab+', 'ab+'): 1,
 ('ab+', 'ab-'): 0,
 ('ab+', 'b+'): 0,
 ('ab+', 'b-'): 0,
 ('ab+', 'o+'): 0,
 ('ab+', 'o-'): 0,
 ('ab-', 'a+'): 0,
 ('ab-', 'a-'): 0,
 ('ab-', 'ab+'): 1,
 ('ab-', 'ab-'): 1,
 ('ab-', 'b+'): 0,
 ('ab-', 'b-'): 0,
 ('ab-', 'o+'): 0,
 ('ab-', 'o-'): 0,
 ('b+', 'a+'): 0,
 ('b+', 'a-'): 0,
 ('b+', 'ab+'): 1,
 ('b+', 'ab-'): 0,
 ('b+', 'b+'): 1,
 ('b+', 'b-'): 0,
 ('b+', 'o+'): 0,
 ('b+', 'o-'): 0,
 ('b-', 'a+'): 0,
 ('b-', 'a-'): 0,
 ('b-', 'ab+'): 1,
 ('b-', 'ab-'): 1,
 ('b-', 'b+'): 1,
 ('b-', 'b-'): 1,
 ('b-', 'o+'): 0,
 ('b-', 'o-'): 0,
 ('o+', 'a+'): 1,
 ('o+', 'a-'): 0,
 ('o+', 'ab+'): 1,
 ('o+', 'ab-'): 0,
 ('o+', 'b+'): 1,
 ('o+', 'b-'): 0

In [ ]:
if __name__ == "__main__":
  # Example usage of the can_donate function
  for donor in range(8):
      for recipient in range(8):
          #if can_donate(list(blood_type_encoding.keys())[donor], list(blood_type_encoding.keys())[recipient]) == "new function":
              if can_donate(list(blood_type_encoding.keys())[donor], list(blood_type_encoding.keys())[recipient]): ## if -||- == our new function ##
                  print(f"Both functions agree that: {list(blood_type_encoding.keys())[donor]} can donate to {list(blood_type_encoding.keys())[recipient]}")
              else:
                  print(f"Both functions agree that: {list(blood_type_encoding.keys())[donor]} cannot donate to {list(blood_type_encoding.keys())[recipient]}")
          #else:
          #    print(f"Functions disagree on: {list(blood_type_encoding.keys())[donor]} to {list(blood_type_encoding.keys())[recipient]}")

o- can donate to o-
o- can donate to o+
o- can donate to a-
o- can donate to a+
o- can donate to b-
o- can donate to b+
o- can donate to ab-
o- can donate to ab+
o+ cannot donate to o-
o+ can donate to o+
o+ cannot donate to a-
o+ can donate to a+
o+ cannot donate to b-
o+ can donate to b+
o+ cannot donate to ab-
o+ can donate to ab+
a- cannot donate to o-
a- cannot donate to o+
a- can donate to a-
a- can donate to a+
a- cannot donate to b-
a- cannot donate to b+
a- can donate to ab-
a- can donate to ab+
a+ cannot donate to o-
a+ cannot donate to o+
a+ cannot donate to a-
a+ can donate to a+
a+ cannot donate to b-
a+ cannot donate to b+
a+ cannot donate to ab-
a+ can donate to ab+
b- cannot donate to o-
b- cannot donate to o+
b- cannot donate to a-
b- cannot donate to a+
b- can donate to b-
b- can donate to b+
b- can donate to ab-
b- can donate to ab+
b+ cannot donate to o-
b+ cannot donate to o+
b+ cannot donate to a-
b+ cannot donate to a+
b+ cannot donate to b-
b+ can donate to b+
b